In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
TRAIN_FILE = '../data/tsd_train.csv'
TRIAL_FILE = '../data/tsd_trial.csv'

In [3]:
from transformers import BertTokenizer

In [4]:
train = pd.read_csv(TRAIN_FILE)
trial = pd.read_csv(TRIAL_FILE)

In [5]:
train.head()

,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",What a knucklehead. How can anyone not know th...
4,"[32, 33, 34, 35, 36, 37, 38]","""who do you think should do the killing?""\n\nA..."


In [6]:
train.shape

(7939, 2)

In [7]:
train.isnull().sum()

spans    0
text     0
dtype: int64

In [8]:
train.duplicated().sum()

0

In [9]:
trial.head()

,spans,text
0,"[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",Because he's a moron and a bigot. It's not any...
1,"[29, 30, 31, 32, 33, 34]",How about we stop protecting idiots and let na...
2,"[166, 167, 168, 169, 170, 171]","If people were smart, they would Boycott th..."
3,"[87, 88, 89, 90, 91, 92]",Trump Claimed that Russia will never invade th...
4,[],As long as your willing to pay a lot more for ...


In [10]:
trial.shape

(690, 2)

In [11]:
trial.isnull().sum()

spans    0
text     0
dtype: int64

In [12]:
trial.duplicated().sum()

0

In [13]:
combo = pd.concat([train,trial])

In [14]:
combo.head()

,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",What a knucklehead. How can anyone not know th...
4,"[32, 33, 34, 35, 36, 37, 38]","""who do you think should do the killing?""\n\nA..."


In [15]:
combo.isnull().sum()

spans    0
text     0
dtype: int64

In [16]:
combo.duplicated().sum()

5

In [17]:
combo[combo.duplicated()]

,spans,text
100,"[6, 7, 8, 9, 10]",Trump troll!
224,"[0, 1, 2, 3, 4]",Idiot!
276,"[0, 1, 2, 3, 4]",Idiot
363,"[14, 15, 16, 17, 18, 19]",You can't fix stupid
588,"[11, 12, 13, 14, 15]",You are an idiot.


In [18]:
train[train['text']=='Idiot']

,spans,text
3770,"[0, 1, 2, 3, 4]",Idiot


In [19]:
trial[trial['text']=='Idiot']

,spans,text
276,"[0, 1, 2, 3, 4]",Idiot


In [20]:
merged_df = train.merge(trial, how='inner',indicator=False)

In [21]:
merged_df.shape

(5, 2)

In [22]:
merged_df

,spans,text
0,"[14, 15, 16, 17, 18, 19]",You can't fix stupid
1,"[0, 1, 2, 3, 4]",Idiot!
2,"[0, 1, 2, 3, 4]",Idiot
3,"[6, 7, 8, 9, 10]",Trump troll!
4,"[11, 12, 13, 14, 15]",You are an idiot.


In [23]:
new_train = train[~train.text.isin(merged_df['text'].values)]

In [24]:
new_train.shape

(7934, 2)

In [25]:
new_train.head()

,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",What a knucklehead. How can anyone not know th...
4,"[32, 33, 34, 35, 36, 37, 38]","""who do you think should do the killing?""\n\nA..."


In [26]:
new_train.to_csv('../data/modified_train.csv',index=False)

In [27]:
!pip install termcolor
from termcolor import colored

In [31]:
text = new_train.iloc[5]['text']

In [33]:
spans = eval(new_train.iloc[5]['spans'])

In [34]:
text

'But, but, but, is NOT a defense.  It\'s not even a good deflection.  In America today we have Nazis waving the Nazi flag at rallies in our cities. In what capacity does anyone think this is ok and who would not see that as a problem?\n\nEnough with the "Well what about blah blah" garbage.'

In [35]:
text_list = list(text)

In [37]:
def get_continuous_spans(spans):
    continuous_spans = []
    current_span = []
    for i in range(len(spans)):
        if current_span == []:
            current_span.append(spans[i])
            continue
        if spans[i]==current_span[-1]+1:
            current_span.append(spans[i])
        else:
            continuous_spans.append(current_span)
            current_span = [spans[i]]
    if(current_span!=[]):
        continuous_spans.append(current_span)
    return continuous_spans

Check if any spaces are marked in the toxic spans

In [38]:
def get_space_count(df):
    df['space_count'] = df.apply(lambda x: np.sum([1 if x['text'][i]==' ' else 0 for i in eval(x['spans'])]),axis=1)
    return df['space_count'].sum(), df['space_count'].mean(),  df['space_count'].std()

In [41]:
print(['%.2f'% i for i in get_space_count(new_train)])

['13278.00', '1.67', '7.72']


In [42]:
print(['%.2f'% i for i in get_space_count(trial)])

['830.00', '1.20', '4.48']


Check if any words are cut across spans

In [45]:
def check_if_word_cut(text,contiguous_spans):
    words_cuts = []
    for i in contiguous_spans:
        words_cut = 0
        if i[0]==0 and i[-1]==len(text)-1:
            words_cut = 0
        elif i[0]==0:
            if text[i[-1]]!=' ' and text[i[-1]+1].isalnum():
                words_cut+=1
        elif i[-1]==len(text)-1:
            if text[i[0]]!=' ' and text[i[0]-1].isalnum():
                words_cut +=1
        else:
            if text[i[0]]!=' ' and text[i[0]-1].isalnum():
                words_cut +=1
            if text[i[-1]]!=' ' and text[i[-1]+1].isalnum():
                words_cut +=1
        words_cuts.append(words_cut)
    return words_cuts     

In [28]:
def words_cut_count(df):
    df['contiguous_spans'] = df.apply(lambda x : get_continuous_spans(eval(x['spans'])),axis=1)
    df['words_cut'] = df.apply(lambda x: np.sum(check_if_word_cut(x['text'],x['contiguous_spans'])),axis=1)
    return df['words_cut'].sum(),df['words_cut'].mean(),df['words_cut'].std()

In [48]:
print(['%.2f'% i for i in words_cut_count(new_train)])

['263.00', '0.03', '0.20']


In [49]:
print(['%.2f'% i for i in words_cut_count(trial)])

['26.00', '0.04', '0.23']


In [50]:
def print_contiguous_spans(text,contiguous_spans):
    return [text[i[0]:i[-1]+1] for i in contiguous_spans]

In [52]:
new_train[new_train['words_cut']>0].apply(lambda x: (x['text'],len(x['text']),x['spans'],x['contiguous_spans'],print_contiguous_spans(x['text'],x['contiguous_spans']),x['words_cut']),axis=1).values

      ('Nah!  We can pretend he is a Republican.  Off with his head!  Impeach him!  Recall him! etc, etc, etc.', 102, '[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67]', [[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67]], ['head!  Impeac'], 1.0),
       ("Roof is not really an ignorant redneck or KKK white trash--his family seemed quite well-off when he was a child and according to reports he was bright in school.   He's clearly not normal--his OCD is likely from Asperger's or autism.  He got into drugs and someone like him, very obviously, should not have been permitted to carry a gun.", 337, '[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 

Check if any spans end or start with space

In [61]:
import string
def check_if_start_or_end_with_whitespace(text,contiguous_spans):
    start_or_end_with_space = []
    for i in contiguous_spans:
        s_e = 0
        if text[i[0]] in string.whitespace:
            s_e+=1
        if text[i[-1]] in string.whitespace:
            s_e+=1
        start_or_end_with_space.append(s_e)
    return start_or_end_with_space 

In [62]:
def start_end_with_space(df):
    df['start_or_end_with_space'] = df.apply(lambda x: np.sum(check_if_start_or_end_with_space(x['text'],x['contiguous_spans'])),axis=1)
    return df['start_or_end_with_space'].sum(),df['start_or_end_with_space'].mean(),df['start_or_end_with_space'].std()
    

In [63]:
print(['%.2f'% i for i in start_end_with_space(new_train)])

['20.00', '0.00', '0.05']


In [64]:
print(['%.2f'% i for i in start_end_with_space(trial)])

['1.00', '0.00', '0.04']


In [65]:
new_train[new_train['start_or_end_with_space']>0].apply(lambda x: (x['text'],len(x['text']),x['start_or_end_with_space'],print_contiguous_spans(x['text'],x['contiguous_spans'])),axis=1).values

array([('I am watching Goodale on TV giving statements about this. He is such an ass. What a slime bucket.  i hope he knows the depth of disgust he is creating for himself and his most beloved Justin trudeau. Will the media  hold Trudeau to task as Prime Minister and ask  questions him real questions when he gets back from his Pride Parade across Europe?  Or will they  ask him low ball questions about the weather to save him embarrassment of looking like an idiot.', 459, 1.0, ['He is such an ass', ' i hope he knows the depth of disgust he is creating for himself and his most beloved Justin trudea', 'Or will they  ask him low ball questions about the weather to save him embarrassment of looking like an idiot']),
       ('Most of trump supporters in red states rely on the government in on way or another for support and there is no way they voted for trump to cut those programs.  They want more government support, not less.  \ntrump promises the moon and only stupid people believe him.', 

Clean out spans which are half in, remove trailing and beginning spaces

In [200]:
def find_word_by_character_index(text, idx):
    if(not text[idx].isalnum()):
        backward = idx
        forward = idx
        while(backward>-1 and not text[backward].isalnum()):
            backward-=1
        while(forward<len(text) and text[forward].isalnum()):
            forward+=1

        return text[backward+1:forward], backward+1,forward-1
    else:
        return text[idx], idx, idx

In [201]:
new_train.loc[20,'spans']

'[0, 1, 2, 3, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]'

In [202]:
new_train.loc[20,'text']

'Kill some more kids and then complain about guns, LOL the left is a joke'

In [203]:
find_word_by_character_index(new_train.loc[20,'text'],15)

('kids', 15, 18)

In [276]:
def clean_text(text,contiguous_spans):
    new_contiguous_spans = []

    for i in contiguous_spans:
        start = i[0] 
        end = i[-1]

        if start==0 and end==len(text)-1:
            new_contiguous_spans.append([start,end])

        elif start==0:
            if text[end].isalnum() and text[end+1].isalnum():

                full_word,full_start,full_end = find_word_by_character_index(text,end)
                cut_word_len = end-full_start+1
                if(cut_word_len*2>=len(full_word)):
                    new_contiguous_spans.append([start,full_end])
                else:
                    new_contiguous_spans.append([start,full_start-1])


        elif i[-1]==len(text)-1:
            if text[start].isalnum() and text[start-1].isalnum():
                full_word, full_start,full_end = find_word_by_character_index(text,start)
                cut_word_len = full_end-start+1
                if(cut_word_len*2>=len(full_word)):
                    new_contiguous_spans.append([full_start,end])
                else:
                    new_contiguous_spans.append([full_end+1,end])
                
                
        else:
            new_start = start
            new_end = end
           
            if text[start].isalnum() and text[start-1].isalnum():
                full_word, full_start,full_end = find_word_by_character_index(text,start)
                cut_word_len = full_end-start+1
                if(cut_word_len*2>=len(full_word)):
                    new_start = full_start
                else:
                    new_start = full_end+1

            if text[end].isalnum() and text[end+1].isalnum():
                full_word, full_start,full_end = find_word_by_character_index(text, end)
                cut_word_len = end-full_start+1
                if(cut_word_len*2>=len(full_word)):
                    new_end = full_end
                else:
                    new_end = full_start-1
            new_contiguous_spans.append([new_start,new_end])
    ## Remove Spaces from span beginning and end

    newest_contiguous_spans = []
    for i in new_contiguous_spans:
        start = i[0]
        end = i[-1]
        while start<=end:
            if(not (text[start].isalnum()) or not (text[end].isalnum())):
                if not (text[start].isalnum()):
                    start+=1
                if not (text[end].isalnum()):
                    end-=1
            else:
                break
        if(start<=end):
            newest_contiguous_spans.append([start,end])
    return newest_contiguous_spans


In [277]:
text = "Hello How are you sir, My name is Gunjan Chhablani?"

In [285]:
contiguous_spans = get_continuous_spans([1,2,3,4,6,7,11,12,13,14,15,16,17, 18, 19, 22,23,24,25,30,31,32,33,34,35,36, 41,42,43,44])

In [286]:
print_contiguous_spans(text,contiguous_spans)

['ello', 'Ho', 're you si', ' My ', ' is Gun', 'Chha']

In [287]:
new_contiguous_spans = clean_text(text,contiguous_spans)

In [288]:
print_contiguous_spans(text,new_contiguous_spans)

['Hello', 'How', 'are you sir', 'My', 'is Gunjan']

In [299]:
clean_train = new_train.copy()

In [300]:
clean_train['contiguous_spans'] = clean_train.apply(lambda x: clean_text(x['text'],x['contiguous_spans']),axis=1)

In [301]:
clean_trial = trial.copy()

In [302]:
clean_trial['contiguous_spans'] = clean_trial.apply(lambda x:clean_text(x['text'],x['contiguous_spans']),axis=1)

In [303]:
start_end_with_space(clean_train)

(0.0, 0.0, 0.0)

In [304]:
start_end_with_space(clean_trial)

(0.0, 0.0, 0.0)

In [307]:
def words_cut_count_clean(df):
    df['words_cut'] = df.apply(lambda x: np.sum(check_if_word_cut(x['text'],x['contiguous_spans'])),axis=1)
    return df['words_cut'].sum(),df['words_cut'].mean(),df['words_cut'].std()

In [308]:
words_cut_count_clean(clean_train)

(0.0, 0.0, 0.0)

In [309]:
words_cut_count_clean(clean_trial)

(0.0, 0.0, 0.0)

In [310]:
clean_train.to_csv('../data/clean_train.csv',index=False)

In [311]:
clean_trial.to_csv('../data/clean_trial.csv',index=False)

In [29]:
clean_train = pd.read_csv('../data/clean_train.csv')

In [38]:
clean_trial = pd.read_csv('../data/clean_trial.csv')

In [39]:
def get_spans_from_contiguous(contiguous_spans):
    spans = []
    for i in eval(contiguous_spans):
        spans+=list(range(i[0],i[-1]+1))
    return spans

In [40]:
clean_train['spans'] = clean_train['contiguous_spans'].apply(get_spans_from_contiguous)

In [41]:
clean_trial['spans'] = clean_trial['contiguous_spans'].apply(get_spans_from_contiguous)

In [42]:
clean_train[['spans','text']].to_csv('../data/clean_train.csv',index=False)

In [43]:
clean_trial[['spans','text']].to_csv('../data/clean_trial.csv',index=False)

In [45]:
new_train.to_csv('../data/modified_train.csv',index=False)

Tokens Count

In [312]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize_text(df):
    df['#tokens'] = df.apply(lambda x: len(tokenizer.tokenize(x['text'])),axis=1)
    df['#words'] = df.apply(lambda x: len(x['text'].split()),axis=1)
    

In [314]:
lists = []
for df in [clean_train,clean_trial,new_train,trial]:
    tokenize_text(df)
    lists.append(['%.2f' % i for i in (df['#tokens'].mean(),df['#tokens'].std(),df['#tokens'].max(),df['#tokens'].min())]+['%.2f' % i for i in (df['#words'].mean(),df['#words'].std(),df['#words'].max(),df['#words'].min())])

In [315]:
print(pd.DataFrame(lists, columns=["#Tokens(mean)","#Tokens(std)","#Tokens(max)","#Tokens(min)","#Words(mean)","#Words(std)","#Words(max)","#Words(min)"]).T.to_markdown())

|               |      0 |      1 |      2 |      3 |
|:--------------|-------:|-------:|-------:|-------:|
| #Tokens(mean) |  47.53 |  46.1  |  47.53 |  46.1  |
| #Tokens(std)  |  45.46 |  43.82 |  45.46 |  43.82 |
| #Tokens(max)  | 335    | 234    | 335    | 234    |
| #Tokens(min)  |   1    |   1    |   1    |   1    |
| #Words(mean)  |  35.97 |  35.01 |  35.97 |  35.01 |
| #Words(std)   |  34.97 |  34.42 |  34.97 |  34.42 |
| #Words(max)   | 192    | 182    | 192    | 182    |
| #Words(min)   |   1    |   1    |   1    |   1    |
